In [1]:
import numpy as np
import heston_call_numpy as hc
from mtgl_test import *

In [2]:
# input: calibrated parameters (x0,r,kappa,mu,eta,V0,rho) for each maturity 
# function: generate option payoff given parameters from the Heston model (question: how to deal with running max when we have different calibrated parameters for different maturities?)

In [3]:
# question: how do we calculate the running max? Doubly make sure. Recall that we have a new set of calibrated parameters for each maturity.

In [7]:
# code smoothing kernel
def kernel(rho, x):
    # x can be a matrix or a scalar, rho is a scalar
    return ((rho-1)/2)*(np.float_power(np.abs(x)+1,-rho))

In [4]:
# test 'kernel'
x = np.array([[1,2],[1,2]])
kernel(3,x) # test passed

array([[0.125     , 0.03703704],
       [0.125     , 0.03703704]])

In [2]:
def mtgLoss_vanilla(rho, calibrated_payoff, market_price):
    # group by maturities
    mat_num = len(calibrated_payoff) # number of maturities
    itr, strike_num = calibrated_payoff[0].shape
    vanilla_loss = []
    for i in range(mat_num):
        sum_ls = []
        for j in range(itr):
            # perform element-wise matrix multiplication
            diff = calibrated_payoff[i][j].T.reshape(strike_num,1)-market_price[i][j]
            compute_mat = np.multiply(diff,kernel(rho,diff))  
            sum_ls.append(np.sum(np.multiply(compute_mat,compute_mat)))
        vanilla_loss.append(sum(sum_ls)) # summing loss over all iterations
    vanilla_loss = sum(vanilla_loss) # summing loss over all maturities
    # compute average loss across iterations
    avg_loss = vanilla_loss/(itr*mat_num*strike_num)
    
    return avg_loss

In [9]:
x = hc.Heston_stock(itr = 10, dt = 1/12, timesteps = 12)[0]
strikes_call = np.arange(0.4,1.4, 0.2)
print(strikes_call)
maturities = range(1,6,2)
print(list(maturities))

[0.4 0.6 0.8 1.  1.2]
[1, 3, 5]


In [10]:
result = hc.price_payoff_coupling(x = x, strikes_call = strikes_call, maturities = maturities, itr = 10, dt = 1/12, timesteps = 12)
calibrated_payoff = result[1]
market_price = result[0] # vanilla
rho = 5

mtgLoss_vanilla(rho, calibrated_payoff, market_price)

0.021584485742802834

### Previous Formulation (Garbage Collection)

In [22]:
# write the function for martingale projection loss
def mtgLoss_pair(rho, calibrated_payoff, market_price, summation = False):
    # group by maturities
    mat_num = len(calibrated_payoff[1]) # number of maturities
    vanilla_loss = []
    for i in range(mat_num):
        # perform element-wise matrix multiplication
        diff = calibrated_payoff[1][i]-market_price[1][i]
        compute_mat = np.multiply(diff,kernel(rho,diff))  
        vanilla_loss.append(np.sum(np.multiply(compute_mat,compute_mat)))
    vanilla_loss = sum(vanilla_loss)
    # exotic options
    diff = calibrated_payoff[0]-market_price[0]
    compute_mat = np.multiply(diff,kernel(rho,diff)) 
    exotic_loss = np.sum(np.multiply(compute_mat,compute_mat))
    
    if summation:
        sum_loss = exotic_loss+vanilla_loss
        print("Total loss of vanilla and exotic options: ", sum_loss)
        return sum_loss
    else:
        print("Loss in vanilla: ", vanilla_loss)
        print("Loss in exotic: ", exotic_loss)
        return (exotic_loss, vanilla_loss)

In [38]:
# test 'mtgLoss_vanilla'
strikes_call = np.arange(0.4,1.4, 0.2)
print(strikes_call)
maturities = range(1,12,4)
print(list(maturities))

mat_ls_1 = hc.price_option(strikes_call = strikes_call, maturities = maturities, itr = 100, timesteps = 12)
mat_ls_2 = hc.price_option(strikes_call = strikes_call, maturities = maturities, itr = 100, timesteps = 12)

[0.4 0.6 0.8 1.  1.2]
[1, 5, 9]


In [42]:
# they should be a martingale 
rho = 5
mtgLoss_pair(rho, mat_ls_1, mat_ls_2) # but how do we interpret the numerics?

Loss in vanilla:  0.00035888587606875164
Loss in exotic:  0.0020678052254643584


(0.0020678052254643584, 0.00035888587606875164)

In [ ]:
# train neural network on 59.pt

### Train Neural Network on Call_price59.pt

#### Train on LV Model with four maturities

In [2]:
# perform martingale test
maturities = range(16, 65, 16)
print(list(maturities))
# load data
stock_traj = np.loadtxt("stock_traj_LV.txt")
print(stock_traj)

[16, 32, 48, 64]
[[1.         0.98540115 0.99362361 ... 0.         0.         0.        ]
 [1.         1.03447974 1.03733873 ... 0.         0.         0.        ]
 [1.         1.00770235 1.01471627 ... 0.         0.         0.        ]
 ...
 [1.         0.94418389 0.93262643 ... 0.         0.         0.        ]
 [1.         0.96668428 0.92922485 ... 0.         0.         0.        ]
 [1.         1.02036333 0.98314911 ... 0.         0.         0.        ]]


In [3]:
len(stock_traj)

4000

In [4]:
# select the first monte carlo trial of stock trajectory
stock_traj[0]

array([1.        , 0.98540115, 0.99362361, 0.98363215, 0.96423972,
       0.96748948, 0.96924037, 0.97284675, 0.96043164, 0.96024227,
       0.94904202, 0.92177194, 0.90137631, 0.89315194, 0.8793627 ,
       0.88701153, 0.87845844, 0.87892509, 0.87959081, 0.87997603,
       0.88086188, 0.88328755, 0.88134891, 0.88129324, 0.87875605,
       0.87914455, 0.88008326, 0.87843984, 0.88132733, 0.88003719,
       0.88200146, 0.88274384, 0.88489801, 0.89036494, 0.84676188,
       0.87584907, 0.84927815, 0.92995304, 0.91937727, 0.93407285,
       0.87879932, 0.88721365, 0.90531772, 0.94367898, 0.93375301,
       0.96037841, 0.97252077, 0.98961604, 0.99634331, 0.97216123,
       0.99552751, 1.0323447 , 1.01489592, 1.04148948, 1.09775949,
       1.09987235, 1.13087642, 1.16045153, 1.13652158, 1.132442  ,
       1.18201435, 1.1991272 , 1.21370554, 1.22735965, 1.23391056,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [5]:
strikes_call = np.arange(0.8, 1.21, 0.02)
result = hc.price_payoff_coupling(x = stock_traj, strikes_call = strikes_call, maturities = maturities, itr = 1000, dt = 1/96, timesteps = 96)
calibrated_payoff = result[1]
market_price = result[0] # vanilla
rho = 2

KeyboardInterrupt: 

In [30]:
# Save the list to a file
filename = 'LV_calibrated_payoff.pkl'
with open(filename, 'wb') as file:
    pickle.dump(result[1], file)

In [15]:
mtgLoss_vanilla(rho, calibrated_payoff, market_price)

0.006908011778343283

In [18]:
calibrated_payoff[0].shape

(4000, 21)

In [19]:
len(calibrated_payoff)

4

In [28]:
import pickle

#### Train on LSV Model with two maturities

In [9]:
rho = 5
lbd = -50
ubd = 50
conf = 95

In [10]:
# perform martingale test
maturities = range(16, 33, 16)
print(list(maturities))
# load data
stock_traj = np.loadtxt("LSV_calibrated_stock_traj.txt")
print(stock_traj)

[16, 32]
[[1.         0.95635444 0.96101594 ... 0.         0.         0.        ]
 [1.         0.99152702 0.97971606 ... 0.         0.         0.        ]
 [1.         1.00276649 0.98541278 ... 0.         0.         0.        ]
 ...
 [1.         1.00639749 1.00547409 ... 0.         0.         0.        ]
 [1.         1.02952552 1.0381937  ... 0.         0.         0.        ]
 [1.         1.01098406 0.97410154 ... 0.         0.         0.        ]]


In [11]:
# take the first monte carlo iteration
stock_traj[0]

array([1.        , 0.95635444, 0.96101594, 0.96356452, 0.95237756,
       0.95915115, 0.96436673, 0.96914923, 0.98175144, 0.97802693,
       0.99420196, 1.01122165, 1.02124596, 1.00007153, 1.0054847 ,
       1.01051927, 1.00079477, 0.98420453, 0.99797356, 0.9923262 ,
       0.98565644, 0.99156839, 1.04406202, 1.00005972, 0.98420578,
       1.02549112, 0.9924286 , 1.00559068, 1.00461447, 0.85250759,
       0.85628724, 0.82112503, 0.8429659 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [12]:
strikes_call = np.arange(0.8, 1.21, 0.02)
print(strikes_call)
maturities = range(16, 33, 16)
print(list(maturities))

result = hc.price_payoff_coupling(x = stock_traj, strikes_call = strikes_call, maturities = maturities, itr = 1000, dt = 1/96, timesteps = 96)
calibrated_payoff = result[1]
market_price = result[0] # vanilla
rho = 5

[0.8  0.82 0.84 0.86 0.88 0.9  0.92 0.94 0.96 0.98 1.   1.02 1.04 1.06
 1.08 1.1  1.12 1.14 1.16 1.18 1.2 ]
[16, 32]


In [26]:
def mtgLoss_vanilla(rho, calibrated_payoff, market_price):
    # group by maturities
    mat_num = len(calibrated_payoff) # number of maturities
    itr, strike_num = calibrated_payoff[0].shape
    vanilla_loss = []
    for i in range(mat_num):
        sum_ls = []
        for j in range(itr):
            # perform element-wise matrix multiplication
            diff = calibrated_payoff[i][j].T.reshape(strike_num,1)-market_price[i][j]
            compute_mat = np.multiply(diff,kernel(rho,diff))  
            sum_ls.append(np.sum(np.multiply(compute_mat,compute_mat)))
        vanilla_loss.append(sum(sum_ls)) # summing loss over all iterations
    vanilla_loss = sum(vanilla_loss) # summing loss over all maturities
    # compute average loss across iterations
    avg_loss = vanilla_loss/(itr*mat_num*strike_num)
    
    return avg_loss

In [25]:
calibrated_payoff[0].shape

(4000, 21)

In [16]:
calibrated_payoff

[array([[0.09022291, 0.08123633, 0.07224975, ..., 0.        , 0.        ,
         0.        ],
        [0.04872828, 0.0397417 , 0.03075512, ..., 0.        , 0.        ,
         0.        ],
        [0.08261912, 0.07363254, 0.06464596, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.1086222 , 0.09963562, 0.09064904, ..., 0.        , 0.        ,
         0.        ],
        [0.09364501, 0.08465843, 0.07567186, ..., 0.        , 0.        ,
         0.        ],
        [0.15386085, 0.14487428, 0.1358877 , ..., 0.        , 0.        ,
         0.        ]]),
 array([[0.01930582, 0.01031924, 0.00133267, ..., 0.        , 0.        ,
         0.        ],
        [0.06285648, 0.0538699 , 0.04488332, ..., 0.        , 0.        ,
         0.        ],
        [0.07891711, 0.06993053, 0.06094395, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.16700277, 0.1580162 , 0.14902962, ..., 0.00524435, 0.        ,
         0.        ],
        [0.0

In [17]:
len(calibrated_payoff[0])

4000

In [18]:
len(calibrated_payoff[0][0])

21

In [28]:
vanilla_loss = mtgLoss_vanilla(rho, calibrated_payoff, market_price)
vanilla_loss

0.045578388542720435

In [32]:
result = np.loadtxt('rho5sig1.txt')

In [34]:
bond = cutoff(result,conf)
bond

4.498309681496935

In [35]:
21*4000*2

168000

In [36]:
np.sqrt(168000)*vanilla_loss

18.681580135561337

In [31]:
mtgl_test(rho,calibrated_payoff, market_price,lbd,ubd,conf,result,vanilla_loss)

(0.06445757522822573, True)

In [29]:
#mtgLoss_vanilla(rho, calibrated_payoff, market_price)

1.7637917394323581

In [ ]:
# q. experiment of the sigma (detect the bump)